In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mvexperiment.experiment as experiment
import ipywidgets as widgets
from ipyfilechooser import FileChooser
%matplotlib qt

You have an old version of afmformats installed.


In [2]:
fc = FileChooser('./')
display(fc)

FileChooser(path='C:\Users\mv68b\git\NanoPrepare', filename='', title='', show_hidden=False, select_desc='Sele…

In [3]:
f = open(fc.value,'r')
for line in f:
    if line[:8]=='Time (s)':
        break
com = line.split('\t')
n=[0,com.index('Load (uN)'),com.index('Piezo (nm)'),com.index('Indentation (nm)')]
time=[]
load=[]
z=[]
ind=[]
for line in f:
    cols = line.split('\t')
    time.append(float(cols[n[0]]))
    load.append(float(cols[n[1]]))
    z.append(float(cols[n[2]]))
    ind.append(float(cols[n[3]]))
time=np.array(time)
z=np.array(z)
load=np.array(load)
f.close()

In [4]:
fig,ax=plt.subplots(1,2,figsize=(14,6))

ax[0].plot(time,z)
ax[1].plot(time,load)

In [5]:
f = open(fc.value,'r')
for line in f:
    if 'Profile' in line:
        break
times=[]
for line in f:
    com = line.strip().split(' ')
    if com[0]=='Time':
        times.append(float(com[1]))
    else:
        break
astart=[]
aend=[]
for line in f:
    if 'DMA absolute start times' in line:
        com = line.strip().split('\t')[1]        
        astart = [float(a) for a in com.split(',')]
    elif 'DMA absolute end times' in line:
        com = line[len('DMA absolute end times (s)'):].strip()
        aend = [float(a) for a in com.split(',')]
    elif 'Initial relaxation' in line:
        delta = float(line[line.rfind(' '):].strip())
    elif 'Model' in line:
        break    
f.close()

fig,ax=plt.subplots(1,2,figsize=(14,6),sharex=True)

ax[0].plot(time,z)
ax[1].plot(time,load)

stimes = [np.argmin((time-t)**2) for t in astart]
etimes = [np.argmin((time-t)**2) for t in aend]

istart = np.argmin((time-astart[0]+delta/4)**2)
flat = np.average( z[istart:stimes[0]] )

onesec = np.argmin((time-1)**2)

segments=[]

for j in range(len(stimes)):
    flat = z[stimes[j]]
    area1 = stimes[j]-onesec
    area2 = etimes[j]+onesec
    ax[0].plot([time[area1],time[area2]],[flat,flat],'--',color='C1')
    
    amp = np.min( [np.max(z[area1:area2])-flat,flat-np.min(z[area1:area2]) ]) 
    
    upstart = area1
    while(z[upstart] < flat+amp/2):
        upstart+=1
    while(z[upstart] > flat):
        upstart-=1
    
    upend = area2
    while(z[upend] > flat-amp/2):
        upend-=1
    while(z[upend] < flat):
        upend+=1
    
    segments.append([upstart,upend,flat,time[upstart:upend],z[upstart:upend],load[upstart:upend]])
    
    ax[0].plot(time[upstart],z[upstart],'ro')
    ax[0].plot(time[upend],z[upend],'go')
    ax[1].plot(time[upstart],load[upstart],'ro')
    ax[1].plot(time[upend],load[upend],'go')
        


IndexError: list index out of range